In [ ]:
import numpy as np
import mindspore as ms
from mindspore import Tensor, ops
import mindspore as ms
import sys
sys.path.append('..')
from data_processing import X_train,X_test,y_train,y_test
from DQAS_tool import Mindspore_ansatz2,sampling_from_structure,nmf_gradient,vag_nnp2,zeroslike_grad_nnp,generate_pauli_string
import numpy as np
import json
import tensorcircuit as tc
import tensorflow as tf
from DQAS_tool import best_from_structure,DQAS_accuracy,extract_parameterss
# #Operator Pool


In [2]:

unbound_opeartor_pool = \
["X0 X1 I2 I3 I4 I5 I6 I7", 
 "I0 X1 X2 I3 I4 I5 I6 I7",
 "I0 I1 X2 X3 I4 I5 I6 I7",
 "I0 I1 I2 X3 X4 I5 I6 I7",
 "I0 I1 I2 I3 X4 X5 I6 I7",
 "I0 I1 I2 I3 I4 I4 X6 I7",
 "Y0 Y1 I2 I3 I4 I4 I6 I7",
 "I0 Y1 Y2 I3 I4 I4 I6 I7",
 "I0 I1 Y2 Y3 I4 I4 I6 I7",
 "I0 I1 I2 Y3 Y4 I4 I6 I7",
 "I0 I1 I2 I3 Y4 Y5 I6 I7",
 "I0 I1 I2 I3 I4 Y5 Y6 I7",
 ]
bound_opeartor_pool = \
['ZYIXXXZX',
 'IZZYYXZI',
 'XIIXYIIZ',
 'IIIYYIYX',
 'IZZYIXZI',
 'ZZYYYYYY']

num_layer = 12
# 定义标准差和形状
stddev = 0.02
shape_parametized = len(unbound_opeartor_pool) #12
shape_unparametized = len(bound_opeartor_pool) #6
shape_nnp = (num_layer, shape_parametized)
shape_stp = (num_layer, shape_parametized+shape_unparametized)

nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp)

np.random.seed(10)
unbound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[0] for i in range(shape_parametized)]
bound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[1] for i in range(shape_parametized,shape_parametized+shape_unparametized)]

In [ ]:

#from DQAS_tool import  DQASAnsatz_from_result,DQAS_accuracy
K = tc.set_backend("tensorflow")
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.05))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
verbose = False
# 设置超参数
epochs = 30
batch_size=50
shape_nnp = (num_layer, shape_parametized)
shape_stp = (num_layer, shape_parametized+shape_unparametized)
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(np.float64)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(np.float64)
print(stp.shape)
avcost1 = 0

ops_onehot = ops.OneHot(axis=-1)

batch_loss_history=[] # 记录每个epoch的batch_size损失值
structure_distribution_history=[] # 记录每个epoch的结构参数
ansatz_params_history=[] # 记录每个epoch的网络参数
best_candidates_history=[] # 记录每个epoch的最佳候选
acc_history = [] #记录每个epoch的准确率


for epoch in range(1):  # 更新结构参数的迭代
    avcost2 = avcost1
    costl = []
    tmp = np.stack([sampling_from_structure(stp,num_layer,shape_parametized) for _ in range(batch_size)])
    batch_structure = ops_onehot(ms.Tensor(tmp),shape_parametized+shape_unparametized,ms.Tensor(1),ms.Tensor(0))
    #print(batch_structure.shape)
    # print(tmp,batch_structure)
    loss_value = []
    grad_nnps = []
    grad_stps = []
    
    for i in batch_structure:
        #print(ops.Argmax()(i))          
        infd, grad_nnp = vag_nnp2(Structure_params=i,
                                  Ansatz_params=nnp,
                                  paramerterized_pool=unbound_opeartor_pool,unparamerterized_pool=bound_opeartor_pool,
                                  num_layer=num_layer,n_qbits=8)(ms.Tensor(X_train),ms.Tensor(y_train))
        
        grad_nnp_zeroslike = zeroslike_grad_nnp(batch_sturcture=i,grad_nnp=grad_nnp[0],shape_parametized=shape_parametized,ansatz_parameters=nnp)
        gs = nmf_gradient(structures=stp,oh=i,num_layer=num_layer,size_pool=stp.shape[1])
        #print(infd,grad_nnp)
        loss_value.append(infd)
        grad_nnps.append(ms.Tensor(grad_nnp_zeroslike,dtype=ms.float64))
        grad_stps.append(gs)
      
    infd = ops.stack(loss_value)
    gnnp = ops.addn(grad_nnps)
    gstp = [(infd[i] - avcost2) * grad_stps[i] for i in range(infd.shape[0])]
    gstp_averge = ops.addn(gstp) / infd.shape[0]
    avcost1 = sum(infd) / infd.shape[0]
    
    gnnp_tf = tf.convert_to_tensor(gnnp.asnumpy(),dtype=tf.float64)
    nnp_tf = tf.convert_to_tensor(nnp,dtype=tf.float64)
    gstp_averge_tf = tf.convert_to_tensor(gstp_averge.reshape(stp.shape).asnumpy(),dtype=tf.float64)
    stp_tf = tf.convert_to_tensor(stp,dtype=tf.float64)
     # 更新参数
    nnp_tf = network_opt.update(gnnp_tf, nnp_tf)
    stp_tf = structure_opt.update(gstp_averge_tf, stp_tf) 
    
    nnp = nnp_tf.numpy()
    stp = stp_tf.numpy()

    batch_loss_history.append(avcost1)
    structure_distribution_history.append(stp)
    ansatz_params_history.append(nnp)
    #best_candidates_history.append(best_from_structure(cand_preset.asnumpy()))
    
    
    
    if epoch % 2 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched平均损失: ",
            avcost1,
        )
    
        if verbose:
            print(
                "strcuture parameter: \n",
                stp,
                "\n network parameter: \n",
                nnp,
            )
        
        cand_preset = best_from_structure(stp)
        best_candidates_history.append(cand_preset)
        print("最好的候选结构:",cand_preset)
        stp_for_test = ops_onehot(ms.Tensor(cand_preset),shape_parametized+shape_unparametized,ms.Tensor(1),ms.Tensor(0))
        test_ansatz = Mindspore_ansatz2(Structure_p=stp_for_test,parameterized_pool=unbound_opeartor_pool,unparameterized_pool=bound_opeartor_pool,num_layer=num_layer,n_qbits=8)
        
        ansatz_parameters=[]
        for layerIndex,i in enumerate(cand_preset):
            if i >=len(unbound_opeartor_pool):
                continue
            else:
                ansatz_parameters.append(nnp[layerIndex,i])
        
        acc = DQAS_accuracy(ansatz=test_ansatz,Network_params=ansatz_parameters,n_qbits=8)
        acc_history.append(acc)
        print(f'二分类准确率 Acc ={acc*100}% ')
        

In [1]:
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(1e-3))
epochs = 200
stp_for_test = ops_onehot(ms.Tensor(cand_preset),shape_parametized+shape_unparametized,ms.Tensor(1),ms.Tensor(0))
for epoch in range(epochs):
    infd, grad_nnp = vag_nnp2(Structure_params=stp_for_test,Ansatz_params=nnp,paramerterized_pool=unbound_opeartor_pool,
                              unparamerterized_pool=bound_opeartor_pool,
                              num_layer=num_layer,n_qbits=8)(ms.Tensor(X_train),ms.Tensor(y_train))
    grad_nnp_zeroslike = zeroslike_grad_nnp(batch_sturcture=stp_for_test,grad_nnp=grad_nnp[0],shape_parametized=shape_parametized,ansatz_parameters=nnp)
    
    nnp_tf  = tf.convert_to_tensor(nnp,dtype=tf.float64)
    gnnp_tf = tf.convert_to_tensor(grad_nnp_zeroslike,dtype=tf.float64)
    infd_tf = tf.convert_to_tensor(infd.numpy(),dtype=tf.float64)
    nnp_tf = network_opt.update(gnnp_tf, nnp_tf)
    nnp = nnp_tf.numpy()
    if epoch % 100 == 0 or epoch == epochs - 1:
        print(f'epoch:{epoch},loss:{infd},grad:{grad_nnp[0]}')

NameError: name 'tc' is not defined

In [9]:
def generate_ansatz(stp:np.array):
    stp_for_test = ops_onehot(ms.Tensor(stp),shape_parametized+shape_unparametized,ms.Tensor(1),ms.Tensor(0))
    test_ansatz = Mindspore_ansatz2(Structure_p=stp_for_test,parameterized_pool=unbound_opeartor_pool,unparameterized_pool=bound_opeartor_pool,num_layer=num_layer,n_qbits=8)
    return test_ansatz